Топ водіїв за прибутком за останній тиждень

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder \
    .appName("SimpleApp") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/15 13:36:05 WARN Utils: Your hostname, DESKTOP-DP2FLCF resolves to a loopback address: 127.0.1.1; using 172.22.104.161 instead (on interface eth0)
25/04/15 13:36:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/15 13:36:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/15 13:36:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_fare_init = spark.read.option("header", True).option("inferSchema", True).csv("../data_lake/trip_fare_1.csv")
df_trip_init = spark.read.option("header", True).option("inferSchema", True).csv("../data_lake/trip_data_1.csv")

In [3]:
for column in df_fare_init.columns:
    df_fare_init = df_fare_init.withColumnRenamed(column, column.strip().replace(" ", "_"))

In [4]:
from pyspark.sql.functions import to_date, lit

df_trip = df_trip_init.filter(
    (col("passenger_count") > 0) & (col("passenger_count") <= 10) &
    (col("trip_time_in_secs") > 0) &
    (col("trip_distance") > 0) &
    (col("pickup_latitude").between(40.5, 41.0)) &
    (col("dropoff_latitude").between(40.5, 41.0)) &
    (col("pickup_longitude").between(-74.5, -73.0)) &
    (col("dropoff_longitude").between(-74.5, -73.0)) &
    (col("pickup_datetime") <= lit("2013-01-27"))
)

df_fare = df_fare_init.filter(
    (col("fare_amount") >= 0) &
    (col("tip_amount") >= 0) &
    (col("tolls_amount") >= 0) &
    (col("total_amount") >= 0) &
    ((col("fare_amount") > 0) | (col("payment_type") == "NOC")) &
    (col("pickup_datetime") <= lit("2013-01-27"))
)

In [5]:
from pyspark.sql.functions import max, min, col, to_date, trunc

df = df_trip.join(df_fare, on=["medallion", "hack_license", "pickup_datetime", "vendor_id"])


In [6]:
from pyspark.sql.functions import col, hour, when, to_date, max as Fmax, date_sub, expr, avg, round as Fround
from pyspark.sql.functions import col, to_date, max as Fmax, date_sub, sum as Fsum


In [ ]:
week_start = date_sub(lit("2013-01-27"), 6)

In [8]:
df_last_week = df.withColumn("trip_date", to_date("pickup_datetime")) \
    .filter(col("trip_date") >= week_start)

df_top_drivers = df_last_week.groupBy("hack_license").agg(
    Fround(Fsum("total_amount"), 2).alias("total_earnings")
).orderBy(col("total_earnings").desc())


In [9]:
df_top_drivers.show(20, truncate=False)

25/04/15 13:39:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/15 13:39:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/15 13:39:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/15 13:39:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/15 13:39:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/15 13:39:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/15 13:39:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/15 13:39:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/15 13:39:35 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------------------+--------------+
|hack_license                    |total_earnings|
+--------------------------------+--------------+
|CF4FCE3B4C70CD8639C01A9DFA0ABC36|3716.72       |
|22CA618759C716436EA3257480199A32|3544.41       |
|D85749E8852FCC66A990E40605607B2F|3511.93       |
|895F9C541258B1B18B8EDD4756A845F0|3487.78       |
|7B3DAEAD0556C7DC4BB925B0A8BED5D7|3459.44       |
|6DFAEA426C5CB20249216ED8CC5F5C88|3414.41       |
|80029AAD3540C25106CCF1F42F77E691|3404.38       |
|F788E5B97DBC4DDAD7200A078C2B7FD2|3384.47       |
|F153D0336BF48F93EC3913548164DDBD|3361.66       |
|5649057646D8A96D95C2CD5C846D22D4|3323.36       |
|508B0C200B7911E94E3D58151FADD644|3320.94       |
|9138D1C926529049817DD4609AA26059|3308.89       |
|23F5E8FB4BC7E65E825E8484A596C45C|3216.56       |
|F49FD0D84449AE7F72F3BC492CD6C754|3214.67       |
|A12D0BD80032D6655E8A882D31AB0F3A|3209.71       |
|5403BEF884BCE629B039FAA36D8FA689|3171.22       |
|66AC66EC81245F6D4AE9D2820D49212D|3169.31       |
